In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("1_boot_k3s_resource_usage.csv").drop(columns=["node"])

In [3]:
# change units and convert to int/float
data["cpu"] = data["cpu"].str.replace("m", "").astype(int)
data["memory"] = data["memory"].str.replace("Mi", "").astype(int)
data["cpu_per"] = data["cpu_per"].str.replace("%", "").astype(float) / 100
data["memory_per"] = data["memory_per"].str.replace("%", "").astype(float) / 100

data.head()

,timestamp,cpu,cpu_per,memory,memory_per,service_pods,minio_pods,timescaledb_pods
0,2025-04-21T16:57:49.245164,602,0.07,5888,0.49,1,0,0
1,2025-04-21T16:57:49.245164,365,0.04,4046,0.50,1,0,0
2,2025-04-21T16:58:02.326733,602,0.07,5888,0.49,1,0,0
3,2025-04-21T16:58:02.326733,365,0.04,4046,0.50,1,0,0
4,2025-04-21T16:58:13.930135,580,0.07,5887,0.49,1,0,0


In [4]:
data = data.groupby("timestamp").agg(
    {
        "cpu": "sum",
        "cpu_per": "mean",
        "memory": "sum",
        "memory_per": "mean",
        "service_pods": "mean",
        "minio_pods": "mean",
        "timescaledb_pods": "mean",
    }
).reset_index()

data = data.drop(data.index[120:200]).reset_index(drop=True)
data["time"] = data.index * 10

data.head()

,timestamp,cpu,cpu_per,memory,memory_per,service_pods,minio_pods,timescaledb_pods,time
0,2025-04-21T16:57:49.245164,967,0.055,9934,0.495,1.0,0.0,0.0,0
1,2025-04-21T16:58:02.326733,967,0.055,9934,0.495,1.0,0.0,0.0,10
2,2025-04-21T16:58:13.930135,979,0.055,9928,0.495,1.0,0.0,0.0,20
3,2025-04-21T16:58:25.333345,990,0.055,9933,0.495,1.0,3.0,0.0,30
4,2025-04-21T16:58:37.216705,2785,0.170,10132,0.510,1.0,3.0,0.0,40


In [5]:
# horizontal lines
line_x = {"MinIO": 3, "TimescaleDB": 15, "Service": 29, "Data Collector": 63, "Stress test": 234-(200-120)}

In [6]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()

TIME_RESOLUTION = 10  # seconds

fig.add_trace(
    go.Scatter(
        x=data["time"],
        y=data["cpu"],
        mode="lines",
        name="CPU Usage",
        line=dict(color="blue"),
    )
)

# add horizontal lines
for x_name, x in line_x.items():
    fig.add_trace(
        go.Scatter(
            x=[x*TIME_RESOLUTION, x*TIME_RESOLUTION],
            y=[data["cpu"].min()-50, data["cpu"].max()],
            mode="lines",
            line=dict(color="red", dash="dash"),
        )
    )

    fig.add_annotation(
        x=x*TIME_RESOLUTION,
        y=data["cpu"].max(),
        text=x_name,
        showarrow=False,
        yshift=10,
        font=dict(color="red")
    )

fig.update_layout(
    xaxis_title="Time (s)",
    yaxis_title="CPU Usage (m)",
    height=400,
    width=1200,
    margin=dict(l=10, r=10, t=10, b=10),
    showlegend=False,
    yaxis=dict(
        range=[data["cpu"].min()-50, data["cpu"].max()+250],
    ),
)
fig.show()

In [7]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()

TIME_RESOLUTION = 10  # seconds

fig.add_trace(
    go.Scatter(
        x=data["time"],
        y=data["memory"],
        mode="lines",
        line=dict(color="blue"),
    )
)

# add horizontal lines
for x_name, x in line_x.items():
    fig.add_trace(
        go.Scatter(
            x=[x*TIME_RESOLUTION, x*TIME_RESOLUTION],
            y=[data["memory"].min()-10, data["memory"].max()],
            mode="lines",
            line=dict(color="red", dash="dash"),
        )
    )

    fig.add_annotation(
        x=x*TIME_RESOLUTION,
        y=data["memory"].max(),
        text=x_name,
        showarrow=False,
        yshift=10,
        font=dict(color="red")
    )

fig.update_layout(
    xaxis_title="Time (s)",
    yaxis_title="Memory usage (MiB)",
    height=400,
    width=1200,
    margin=dict(l=10, r=10, t=10, b=10),
    showlegend=False,
    yaxis=dict(
        range=[data["memory"].min()-10, data["memory"].max()+70],
    ),
)
fig.show()